# SmartSim tutorial 1:  getting started
In this notebook, we will walk through the most basic functionalities of SmartSim, such as setting up an experiment running two models, launching it locally, and collecting its results. We will also look at how we can use the `Ensemble` API to run models collectively. 

## 1.1 Running simple models 
The most common way of defining a workflow in SmartSim is through `Experiment`s. An experiment can start and stop a `Model` and check (`poll`) its status at any time. In section *1.2* we will also see how an `Experiment` can be used to run experiments as `Ensemble`s.

We begin by importing the modules we need: `Experiment` and `RunSettings`. `RunSettings` is the object used in SmartSim to define what will be run by a given `Model`. `RunSettings` is the most basic way of defining execution parameters, and will be perfect for executing programs launched locally, i.e. directly by the operating system, without a workload manager. We also import `os`, as we will need to setup the directory where the `Model`s will place their output and error files.

In [2]:
import os
from smartsim import Experiment
from smartsim.settings import RunSettings

Throughout this notebook, we will incrementally build an `Experiment`. Let's start from the simplest case: a single-`Model` example. Our first `Model` will simply print `hello`, using the shell command `echo`.

In [12]:
exp = Experiment(name="tutorial-experiment", launcher="local")

settings_1 = RunSettings(exe="echo", exe_args="hello")
M1 = exp.create_model(name="tutorial-model-1", run_settings=settings_1)

Once the `Model` has been created by the `Experiment`, we can start it. By setting `summary=True`, we can see a summary of the experiment printed before it is effectively launched. The summary will stay for 10 seconds, and it is useful as a last check. If we set `summary=False`, then the experiment would be launched immediately. We also explicitly set `block=True` (even though it is the default), so that  `Experiment.start` waits until the last `Model` has finished before returning: it will act like a job monitor, letting us know if processes run, complete, or fail.

In [13]:
exp.start(M1, block=True, summary=True)



=== LAUNCH SUMMARY ===
Experiment: tutorial-experiment
Experiment Path: /Users/arigazzi/Documents/DeepLearning/smartsim-dev/SmartSim/tutorials/01_getting_started/tutorial-experiment
Launching with: local
# of Ensembles: 0
# of Models: 1
Database: no

=== MODELS ===
model-1
Model Parameters: 
{}
Model Run Settings: 
Executable: /bin/echo
Executable arguments: ['hello']





20:26:49 C02YR4ANLVCJ SmartSim[66295] INFO model-1(66934): Completed


The model has completed. Let's look at the content of the current working directory.

In [14]:
os.listdir('.')

outputfile = './tutorial-model-1.out'
errorfile = './tutorial-model-1.err'

print("Content of tutorial-model-1.out:")
with open(outputfile, 'r') as fin:
    print(fin.read())
print("Content of tutorial-model-1.err:")
with open(errorfile, 'r') as fin:
    print(fin.read())

Content of model-1.out:
hello

Content of model-1.err:



We can see that two files, `tutorial-model-1.out` and `tutorial-model-1.err` have been created. The `.out` file contains the output generated by `model-1`, and the `.err` file would contain the error messages generated by it. Since there were no errors, the `.err` file is empty.

Now let's run two different `Model` instances at the same time. This is just as easy as running one `Model`, and takes the same steps. This time, we will skip the summary. For each `Model`, we create a `RunSettings` object: it is recommended to always create separate `RunSettings` objects for each `Model`.

In [22]:
exp = Experiment(name="tutorial-experiment", launcher="local")

run_settings_1 = RunSettings("sleep", "3")
run_settings_2 = RunSettings("sleep", "5")
model_1 = exp.create_model("tutorial-model-1", run_settings_1)
model_2 = exp.create_model("tutorial-model-2", run_settings_2)
exp.start(model_1, model_2)

20:35:41 C02YR4ANLVCJ SmartSim[66295] INFO tutorial-model-1(67088): Completed
20:35:41 C02YR4ANLVCJ SmartSim[66295] INFO tutorial-model-2(67089): Running
20:35:42 C02YR4ANLVCJ SmartSim[66295] INFO tutorial-model-1(67088): Completed
20:35:42 C02YR4ANLVCJ SmartSim[66295] INFO tutorial-model-2(67089): Completed


Again, we can check the content of the output and error files.

In [23]:
outputfile = './tutorial-model-1.out'
errorfile = './tutorial-model-1.err'

print("Content of tutorial-model-1.out:")
with open(outputfile, 'r') as fin:
    print(fin.read())
print("Content of tutorial-model-1.err:")
with open(errorfile, 'r') as fin:
    print(fin.read())

outputfile = './tutorial-model-2.out'
errorfile = './tutorial-model-2.err'

print("Content of tutorial-model-2.out:")
with open(outputfile, 'r') as fin:
    print(fin.read())
print("Content of tutorial-model-2.err:")
with open(errorfile, 'r') as fin:
    print(fin.read())

Content of tutorial-model-1.out:

Content of tutorial-model-1.err:

Content of tutorial-model-2.out:

Content of tutorial-model-2.err:



In many cases, a launcher different from `local` can be useful. For example, if `mpirun` is installed on the system, we can run a model through it, by specifying it as `run_command` in `RunSettings`. Since `mpirun` takes arguments (e.g. to define how many processes will be run), we pass them by defining `run_args` in `RunSettings`.

In [35]:
exp = Experiment("tutorial", launcher="local")
run_settings = RunSettings("echo",
                           "hello world!",
                           run_command="mpirun",
                           run_args={"-np": 2}) # note that for base ``RunSettings`` run_args passed literally
                      
model = exp.create_model("tutorial-model-mpirun", run_settings)
exp.start(model, summary=True)



=== LAUNCH SUMMARY ===
Experiment: tutorial
Experiment Path: /Users/arigazzi/Documents/DeepLearning/smartsim-dev/SmartSim/tutorials/01_getting_started/tutorial
Launching with: local
# of Ensembles: 0
# of Models: 1
Database: no

=== MODELS ===
tutorial-model-mpirun
Model Parameters: 
{}
Model Run Settings: 
Executable: /bin/echo
Executable arguments: ['hello', 'world!']
Run Command: mpirun
Run arguments: {'-np': 2}




21:20:26 C02YR4ANLVCJ SmartSim[66295] INFO tutorial-model-mpirun(67710): Completed


This time, since we passed `-np 2` to `mpirun`, in the output file we should find the line `hello world!` twice.

In [36]:
outputfile = './tutorial-model-mpirun.out'
errorfile = './tutorial-model-mpirun.err'

print("Content of tutorial-model-mpirun.out:")
with open(outputfile, 'r') as fin:
    print(fin.read())
print("Content of tutorial-model-mpirun.err:")
with open(errorfile, 'r') as fin:
    print(fin.read())

Content of tutorial-model-mpirun.out:
hello world!
hello world!
hello world!
hello world!
hello world!
hello world!
hello world!
hello world!

Content of tutorial-model-mpirun.err:



## 1.2 Creating and running ensembles of models
In the previous example, the two `Model` instances were created separately. For few, simple `Model`s, this is OK, but what if we needed to run a large number of models, which only differ for some parameter? Defining and adding each one separately would be tedious. For such cases, we will rely on an `Ensemble` of models.  